In [7]:
# step 1: download messages and chats from db
import os

from dotenv import load_dotenv
from pymongo import MongoClient

load_dotenv()

MONGO_CONN_STR = os.getenv("MONGO_CONN_STR")
client = MongoClient(MONGO_CONN_STR)
db_name = os.getenv("MONGO_DB_NAME")
db = client[db_name]
messages_collection = db.telegram_messages
chats_collection = db.telegram_chats

messages_data = messages_collection.find()
chats_data = chats_collection.find()
from telegram_downloader.data_model import ChatData

messages = list(messages_data)
# for msg in messages_data:
#     msg.pop("_id")
#     msg.pop("_")
#     messages.append(Message(**msg))

chats = []
for chat in chats_data:
    chat.pop("_id")
    chats.append(ChatData(**chat))


Idea 1: My messages
- total messages sent by me
- chats with most messages sent
- groups with most messages sent

- plot of my message distrubution per chat
- plot of my message distribution per day, month, year

In [10]:
# import pandas as pd


def flatten_item_recursive(item: dict, prefix: str = "") -> dict:
    if isinstance(item, dict):
        res = {}
        for k, v in item.items():
            new_prefix = f"{prefix}.{k}" if prefix else k
            res.update(flatten_item_recursive(v, prefix=new_prefix))
        return res
    else:
        return {prefix: item}


x = flatten_item_recursive(messages[0])
print(len(x), len(messages[0]))

# def build_messages_db(messages: List[dict])-> pd.DataFrame:

82 41


In [ ]:
my_messages = [msg for msg in messages_data if msg["from_id"] == "petrlavrov"]

In [29]:
import pandas as pd

sample = messages[:100]
df = pd.DataFrame(map(flatten_item_recursive, sample))


In [24]:
# filter out my messages only
user_id = int(os.getenv("TELEGRAM_USER_ID"))
my_messages = [msg for msg in messages if
               (msg.get('from_id', {}) or {}).get('user_id') == user_id]
flatten_messages = list(map(flatten_item_recursive, my_messages))


In [47]:
df = pd.DataFrame(flatten_messages)

In [48]:
chat_names = {chat.id: chat.name for chat in chats}
dict(list(chat_names.items())[:10])
chat_categories = {chat.id: chat.entity_category for chat in chats}


def augment_df(df):
    # chat id
    # chat name
    # chat type -> entity category
    df['chat_id'] = df[['peer_id.channel_id', 'peer_id.chat_id',
                        'peer_id.user_id']].bfill(axis=1).iloc[:, 0].astype(int)
    df['chat_name'] = df['chat_id'].map(chat_names)
    df['chat_category'] = df['chat_id'].map(chat_categories)
    return df


df = augment_df(df)


In [49]:
df['chat_category']


0         group
1         group
2         group
3           bot
4           bot
          ...  
131618    group
131619    group
131620    group
131621    group
131622    group
Name: chat_category, Length: 131623, dtype: object

: 

Awesome! Let's do stats!

Idea 1: My messages
- total messages sent by me
  
- chats with most messages sent
- groups with most messages sent

- plot of my message distrubution per chat
- plot of my message distribution per day, month, year

{2373457856: 'CalmMage Daily Meditation [2373457856]',
 2332193270: 'EF Once-a-week Challenges [2332193270]',
 2066938266: 'Интересные мысли (без репостов) [2066938266]',
 2084292315: 'CalmMage Raw (ex. Daily) [2084292315]',
 1551656064: 'CalmMage Life [1551656064]',
 1722020175: 'EF Channel [1722020175]',
 1867177448: 'CalmMage Dev @calmMageDev [1867177448]',
 1980795916: 'EF: Org [1980795916]',
 6854055412: 'Petr Lavrov Urgent @petrlavrovurgent [6854055412]',
 463004712: 'Anna Lavrova @annlavrova [463004712]'}

In [25]:
# part 0
stats = {}
# part 1: calc stats
stats['total_messages'] = len(df)






131623